In [1]:
import numpy as np
import ten
import ipywidgets as widgets

$$
W = \begin{pmatrix}
\sqrt{\cosh(1/t)} & \sqrt{\sinh(1/t)} \\
\sqrt{\cosh(1/t)} & -\sqrt{\sinh(1/t)}
\end{pmatrix}
$$

In [2]:
def W(temp: float, dtype=np.float32) -> np.ndarray:
	return np.array([
		[np.sqrt(np.cosh(1/temp)), np.sqrt(np.sinh(1/temp))],
		[np.sqrt(np.cosh(1/temp)), -np.sqrt(np.sinh(1/temp))],
	], dtype=dtype)

$$
T_{lrud} = \sum_\alpha W_{\alpha,l} W_{\alpha,r} W_{\alpha,u} W_{\alpha,d}
$$

In [3]:
def T(temp: float, dtype=np.float32) -> np.ndarray:
	w = W(temp, dtype)
	return np.einsum("al,ar,au,ad->lrud", w, w, w, w)

In [4]:
temperature = widgets.FloatSlider(min=2, max=5)
temperature

FloatSlider(value=2.0, max=5.0, min=2.0)

In [48]:
Dslider = widgets.IntSlider(min=2, max=16)

class SliderValue:
	def __init__(self,slider):
		self.slider = slider
	def __get__(self, obj, type=None):
		return self.slider.value
	def __int__(self):
		return self.slider.value

D = SliderValue(Dslider)
Dslider

IntSlider(value=2, max=16, min=2)

In [49]:
t = T(temperature.value)
t

array([[[[2.5430813 , 0.        ],
         [0.        , 1.1752013 ]],

        [[0.        , 1.1752013 ],
         [1.1752013 , 0.        ]]],


       [[[0.        , 1.1752013 ],
         [1.1752013 , 0.        ]],

        [[1.1752013 , 0.        ],
         [0.        , 0.54308057]]]], dtype=float32)

In [50]:
def M(tensor: np.ndarray) -> np.ndarray:
	m = np.einsum("abci,deif->adbecf", tensor, tensor)
	return m.reshape((m.shape[0]*m.shape[1], m.shape[2]*m.shape[3], m.shape[4], m.shape[5]))

In [51]:
m = M(t)
S, Us = ten.hosvd(m)

In [54]:
eps1 = sum(np.sum(np.square(np.abs(S[i,:]))) for i in range(int(D), S.shape[0]))
eps1

10.105166673660278

In [55]:
eps2 = sum(np.sum(np.square(np.abs(S[:,j,:]))) for j in range(int(D), S.shape[1]))
eps2

10.105167388916016

In [56]:
# TODO renormalize
if eps1 < eps2:
	S = S[0:int(D), :]
	Us[0] = Us[0][:,0:int(D)]
else:
	S = S[:, 0:int(D), :]
	Us[1] = Us[1][:,0:int(D)]

(2, 4, 2, 2)